In [ ]:
from geopandas import read_file
import numpy as np
from sqlalchemy import create_engine,text

# Replace with enacit4r-tiles postgres credentials (enactit4r keeweb)
dbuser = 'postgres'
dbpassword = 'password'
dbhost = 'localhost:25432'
# dbhost = 'enacit4r-tiles.epfl.ch:25432'
dbname = 'db'
engine = create_engine(f"postgresql://{dbuser}:{dbpassword}@{dbhost}/{dbname}")  
schema = 'monash'

input_folder = '~/projects/monash_data/work/original/FloodMaps_V2_ 18102023/18102023_Flood Maps/GIS files'

In [ ]:
def write_to_db(ari_input, table, label_field):
    ari_input['cc']['100'].rename(columns={'gridcode': 'code_100_cc'}, inplace=True)
    ari_input['cc']['100'].drop(columns=['Id', label_field], inplace=True)
    ari_input['cc']['50'].rename(columns={'gridcode': 'code_50_cc'}, inplace=True)
    ari_input['cc']['50'].drop(columns=['Id', label_field], inplace=True)
    ari_input['cc']['20'].rename(columns={'gridcode': 'code_20_cc'}, inplace=True)
    ari_input['cc']['20'].drop(columns=['Id', label_field], inplace=True)

    ari_input['nocc']['100'].rename(columns={'gridcode': 'code_100_base'}, inplace=True)
    ari_input['nocc']['100'].drop(columns=['Id', label_field], inplace=True)
    ari_input['nocc']['50'].rename(columns={'gridcode': 'code_50_base'}, inplace=True)
    ari_input['nocc']['50'].drop(columns=['Id', label_field], inplace=True)
    ari_input['nocc']['20'].rename(columns={'gridcode': 'code_20_base'}, inplace=True)
    ari_input['nocc']['20'].drop(columns=['Id', label_field], inplace=True)

    aricc = ari_input['cc']['100'].overlay(ari_input['cc']['50'], how='union', keep_geom_type=True).overlay(ari_input['cc']['20'], how='union', keep_geom_type=True)
    arinocc = ari_input['nocc']['100'].overlay(ari_input['nocc']['50'], how='union', keep_geom_type=True).overlay(ari_input['nocc']['20'], how='union', keep_geom_type=True)
    ari = arinocc.overlay(aricc, how='union', keep_geom_type=True)

    for key in ['code_100_cc', 'code_50_cc', 'code_20_cc', 'code_100_base', 'code_50_base', 'code_20_base']:
        ari[key] = ari[key].apply(lambda x: 0 if np.isnan(x) else int(x))
    
    ari.to_postgis(table, engine, if_exists='replace', schema=schema, index=True, index_label='fid')
    with engine.connect() as connection:
        connection.execute(text(f"ALTER TABLE {schema}.{table} ADD PRIMARY KEY (fid);"))
        connection.commit()

In [ ]:
ari_input = {
  'cc': {
    '100': read_file(f'{input_folder}/Risk_v2/ari100cc_risk.shp'),
    '50': read_file(f'{input_folder}/Risk_v2/ari50cc_risk.shp'),
    '20': read_file(f'{input_folder}/Risk_v2/ari20cc_risk.shp'),
  },
  'nocc': {
    '100': read_file(f'{input_folder}/Risk_v2/ari100nocc_risk.shp'),
    '50': read_file(f'{input_folder}/Risk_v2/ari50nocc_risk.shp'),
    '20': read_file(f'{input_folder}/Risk_v2/ari20nocc_risk.shp'),
  }
}

write_to_db(ari_input, 'ari_risk', 'Risk')

In [ ]:
ari_input = {
  'cc': {
    '100': read_file(f'{input_folder}/Hazard_v2/ari100cc_hzd.shp'),
    '50': read_file(f'{input_folder}/Hazard_v2/ari50cc_hzrd.shp'),
    '20': read_file(f'{input_folder}/Hazard_v2/ari20cc_hzrd.shp'),
  },
  'nocc': {
    '100': read_file(f'{input_folder}/Hazard_v2/ari100nocc_hzd.shp'),
    '50': read_file(f'{input_folder}/Hazard_v2/ari50nocc_hzd.shp'),
    '20': read_file(f'{input_folder}/Hazard_v2/ari20nocc_hzd.shp'),
  }
}

write_to_db(ari_input, 'ari_hzd', 'Hazard')